Before getting started check the allocated GPU spec

If the output is `/bin/bash: line 1: nvidia-smi: command not found`, then it probably means you are not using a GPU runtime on Colab.

**Change runtime type**. Select T4.

If all goes well, the command should be similar to `GPU 0: Tesla T4 (UUID: GPU-1accef80-292a-5b97-260d-f3a3cfc2840d)`

In [2]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


In [1]:
%%capture
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score

In [2]:
import torch;
# assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# CMAKE_ARGS="-DLLAMA_METAL=on"  FORCE_CMAKE=1 pip install llama-cpp-python==0.1.83 --no-cache-dir
device = torch.device('cpu')

ImportError: dlopen(/Users/ccugutrillague/mambaforge/envs/coderefinery/lib/python3.11/site-packages/torch/_C.cpython-311-darwin.so, 0x0002): Library not loaded: @rpath/libtorch_cpu.dylib
  Referenced from: <2149917B-77FA-3040-941F-4F37575012EB> /Users/ccugutrillague/mambaforge/envs/coderefinery/lib/python3.11/site-packages/torch/lib/libtorch_python.dylib
  Reason: tried: '/Users/ccugutrillague/mambaforge/envs/coderefinery/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib' (no such file), '/Users/runner/work/_temp/anaconda/envs/wheel_py311/lib/libtorch_cpu.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/runner/work/_temp/anaconda/envs/wheel_py311/lib/libtorch_cpu.dylib' (no such file), '/Users/runner/work/_temp/anaconda/envs/wheel_py311/lib/libtorch_cpu.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/runner/work/_temp/anaconda/envs/wheel_py311/lib/libtorch_cpu.dylib' (no such file), '/Users/ccugutrillague/mambaforge/envs/coderefinery/lib/pyt

In [6]:
import torch
tensor_float = torch.rand((1, 1), dtype=torch.float32, device=device)
try:
    result = tensor_float * tensor_float + tensor_float.abs()
except Exception as e:
    print("Exception occurred:", str(e))
else:
    print("Fused Multiplication Successful!")

Fused Multiplication Successful!


In [7]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: ··········
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
huggingface_dataset_name = "neil-code/dialogsum-test"
dataset = load_dataset(huggingface_dataset_name)

In [6]:
install_dataset = load_dataset("carlosug/ResearchInstall")
install_dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['software', 'repo_name', 'readme_url', 'portal', 'stars', 'selection', 'categories', 'Unnamed: 7', 'date_collection', 'date_submission', 'content', 'plan', 'steps', 'seq_order', 'optional_steps', 'extra_info_optional'],
        num_rows: 39
    })
})

In [8]:
dataset["train"][:1]

{'id': ['train_0'],
 'dialogue': ["#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor."],
 'summary': ["Mr. S

In [7]:
# install_dataset = install_dataset.filter(
#     lambda x: (x["steps"] != None)
# )
# install_dataset["train"][:1]

Filter:   0%|          | 0/39 [00:00<?, ? examples/s]

{'software': ['ensuring-trustworthy-and-ethical-behaviour-in'],
 'repo_name': ['https://paperwithcode.com/paper/'],
 'readme_url': ['https://raw.githubusercontent.com/AAAI-DISIM-UnivAQ/DALI/master/README.md'],
 'portal': ['machine_learning'],
 'stars': [15.0],
 'selection': ['latest'],
 'categories': [None],
 'Unnamed: 7': [None],
 'date_collection': ['13/02/2024'],
 'date_submission': ['12/2/24'],
 'content': ['## Installation\n\n**OS X & Linux:**\n1.  To download and install SICStus Prolog (it is needed), follow the instructions at https://sicstus.sics.se/download4.html.\n2.  Then, you can download DALI and test it by running an example DALI MAS:\n```sh\ngit clone https://github.com/AAAI-DISIM-UnivAQ/DALI.git\ncd DALI/Examples/advanced\nbash startmas.sh\n```\n&nbsp;&nbsp;&nbsp;&nbsp; You will see different windows opening:\n* &nbsp;&nbsp;&nbsp;&nbsp; Prolog LINDA server (active_server_wi.pl)\n* &nbsp;&nbsp;&nbsp;&nbsp; Prolog FIPA client (active_user_wi.pl) \n* &nbsp;&nbsp;&nbsp;&nbs

In [9]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [10]:
model_name='microsoft/phi-2'
# device_map = {"": 0} with GPU
original_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                      device_map="auto", # device_map with cuda
                                                      # quantization_config=bnb_config, cuda GPU option
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


**Test the Model with Zero Shot Inferencing¶**

In [37]:
%%time
from transformers import set_seed
seed = 42
set_seed(seed)

index = 10

prompt = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']
topic = dataset['test'][index]['topic']

prompt = f"INSTRUCT: Given a research software, identify the plans and list of tasks. The output should indicate the sequential order in JSON file:\n{prompt}\nOutput:\n"
inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'BASELINE HUMAN TOPIC:\n{topic}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
INSTRUCT: Given a research software, identify the plans and list of tasks. The output should indicate the sequential order in JSON file:
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
Output:

------------------------------------------------------------------------

In [ ]:
%%time
from transformers import set_seed
seed = 42
set_seed(seed)

index = 10

content = install_dataset['train'][index]['content']
steps = install_dataset['train'][index]['steps']
plan = install_dataset['train'][index]['plan']
software = install_dataset['train'][index]['software']


prompt = f"""
Summarize the Instructions of the following {software} Software:

{content}

Instructions:
"""

# prompt = f"INSTRUCT: Given a research software, identify the plans and list of tasks. The output should indicate the sequential order in JSON file:\n{prompt}\nOutput:\n"
inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN STEPS:\n{steps}\n')
print(dash_line)
print(f'BASELINE HUMAN PLAN:\n{plan}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

In [ ]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [65]:
from transformers import AutoModelForSeq2SeqLM
model_baseline='google/flan-t5-base'
second_model = AutoModelForSeq2SeqLM.from_pretrained(model_baseline, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_baseline)

In [ ]:
# seed = 42
# set_seed(seed)

index = 20

content = install_dataset['train'][index]['content']
steps = install_dataset['train'][index]['steps']
plan = install_dataset['train'][index]['plan']
software = install_dataset['train'][index]['software']


prompt = f"""
Summarize the Instructions of the following {software} Software:

{content}

Instructions:
"""

# prompt = f"INSTRUCT: Given a research software, identify the plans and list of tasks. The output should indicate the sequential order in JSON file:\n{prompt}\nOutput:\n"
inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    second_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN STEPS:\n{steps}\n')
print(dash_line)
print(f'BASELINE HUMAN PLAN:\n{plan}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

**Perform Full Fine-Tuning**

convert the readme-instructions (prompt-response) pairs into explicit instructions for the LLM. Prepend an instruction to the start of the dialog with `Extract the instructions from the following software` and to the start of the instructions with `Steps` as follows:




Training prompt (content README)
```md
Extract the instructions from the following software:

  - ## Setup
  - Install the requirements with:
    conda create -n lwm python=3.10
    pip install -U "jax[cuda12_pip]==0.4.23" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
    pip install -r requirements.txt
```

Training response (steps):

```md
Instructions:
    Step1: n.
    Step2: n2.
```


In [1]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])



NameError: name 'dataset' is not defined

In [71]:
def tokenize_function_install(example):
    start_prompt = 'Give me the instructions of the following {software}.\n\n'
    end_prompt = '\n\Instructions: '
    prompt = [start_prompt + content + end_prompt for content in example["content"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["plan"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = install_dataset.map(tokenize_function, batched=True)
# tokenized_datasets = install_dataset.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

KeyError: 'dialogue'

In [33]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Filter:   0%|          | 0/1999 [00:00<?, ? examples/s]

Filter:   0%|          | 0/499 [00:00<?, ? examples/s]

Filter:   0%|          | 0/499 [00:00<?, ? examples/s]

In [34]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (20, 2)
Validation: (5, 2)
Test: (5, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 20
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
})
